# Tensor contraction

Suppose we want to evaluate the following tensor contraction:

![title](../Img/02/02_01.png)

That is we have to contract the tensors with the same indices (or legs):

T<sup>&alpha;</sup><sub>&beta;</sub> = A<sup>&delta;</sup><sub>&gamma;</sub>B<sup>&gamma;&alpha;</sup><sub>&mu;</sub>C<sup>&mu;</sup><sub>&delta;&beta;</sub>

For now, we will not take into account the arrow direction so tensors will just be simple multidimensional arrays. Notice however that in physical context (e.g. bras, kets, symmetries) the arrow direction is important.

In [1]:
# Include the linear algebra module
using LinearAlgebra

First of all we, to show the method, we will consider the bond dimensions all equal to D = 2

In [2]:
Da = 2; # α dimension
Db = 2; # β dimension
Dg = 2; # γ dimension
Dd = 2; # δ dimension
Dm = 2; # μ dimension

Next generate random tensors:

In [3]:
A = rand(Dg,Dd) # Tensor A(γ,δ)

2×2 Array{Float64,2}:
 0.4456    0.243103
 0.555302  0.122862

In [4]:
B = rand(Da,Dm,Dg) # Tensor B(α,μ,γ)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.0472228  0.338458
 0.264855   0.823525

[:, :, 2] =
 0.723074  0.187905
 0.17044   0.343948

In [5]:
C = rand(Db,Dm,Dd) # Tensor C(β,μ,δ)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.718717  0.194719
 0.457629  0.890089

[:, :, 2] =
 0.270973  0.929774
 0.743945  0.656962

### Contraction of C and B

Suppose that we want to contract C and B. In order to do so with standard linear algebra, we need to transform C and B into matrices. Notice that C and B are contracted via the &mu; index.

So the steps are:

In [6]:
# Permute the indices of B so that:
# B(α,μ,γ) -> B(α,γ,μ)

B1 = permutedims(B,[1,3,2])

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.0472228  0.723074
 0.264855   0.17044 

[:, :, 2] =
 0.338458  0.187905
 0.823525  0.343948

In [7]:
# Reshape the B tensor so that it is a matrix
# B(α,γ,μ) --> B(α*γ,μ)

B1 = reshape(B1,(Da*Dg,Dm))

4×2 Array{Float64,2}:
 0.0472228  0.338458
 0.264855   0.823525
 0.723074   0.187905
 0.17044    0.343948

In [8]:
# Permute the indices of C so that:
# C(β,μ,δ) -> C(μ,β,δ)

C1 = permutedims(C,[2,1,3])

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.718717  0.457629
 0.194719  0.890089

[:, :, 2] =
 0.270973  0.743945
 0.929774  0.656962

In [9]:
# Reshape the C tensor so that it is a matrix
# C(μ,β,δ) --> C(μ,β*δ)

C1 = reshape(C1,(Dm,Db*Dd))

2×4 Array{Float64,2}:
 0.718717  0.457629  0.270973  0.743945
 0.194719  0.890089  0.929774  0.656962

In [10]:
# Contract B and C via matrix multiplication

BC = B1*C1

4×4 Array{Float64,2}:
 0.099844  0.322869  0.327486  0.257485
 0.350711  0.854216  0.83746   0.738061
 0.556275  0.498152  0.370642  0.661374
 0.189471  0.384143  0.365978  0.352759

We now have a matrix with combined indices BC(&alpha;&gamma;,&beta;&delta;) and we reshape it into a tensor BC(&alpha;,&gamma;,&beta;,&delta;)

In [11]:
# Reshape the BC matrix so that it is a tensor
# BC(α*γ,β*δ) --> BC(α,γ,β,δ)

BC = reshape(BC,(Da,Dg,Db,Dd))

2×2×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.099844  0.556275
 0.350711  0.189471

[:, :, 2, 1] =
 0.322869  0.498152
 0.854216  0.384143

[:, :, 1, 2] =
 0.327486  0.370642
 0.83746   0.365978

[:, :, 2, 2] =
 0.257485  0.661374
 0.738061  0.352759

We now have to contract the BC tensor with the A matrix (rank-2 tensor).
In order to do so we reshape BC into a matrix and A into a vector.

In [12]:
# Permute BC:
# BC(α,γ,β,δ) --> BC(α,β,γ,δ)

BC = permutedims(BC,[1,3,2,4])

2×2×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.099844  0.322869
 0.350711  0.854216

[:, :, 2, 1] =
 0.556275  0.498152
 0.189471  0.384143

[:, :, 1, 2] =
 0.327486  0.257485
 0.83746   0.738061

[:, :, 2, 2] =
 0.370642  0.661374
 0.365978  0.352759

In [13]:
# Reshape the BC tensor so that it is a matrix
# BC(α,β,γ,δ) --> BC(α*β,γ*δ)

BC = reshape(BC,(Da*Db,Dg*Dd))

4×4 Array{Float64,2}:
 0.099844  0.556275  0.327486  0.370642
 0.350711  0.189471  0.83746   0.365978
 0.322869  0.498152  0.257485  0.661374
 0.854216  0.384143  0.738061  0.352759

In [14]:
# Reshape A into a vector

A1 = A[:]

4-element Array{Float64,1}:
 0.4455998024282619 
 0.5553020192027764 
 0.2431026908697531 
 0.12286247821736929

In [15]:
# Contract BC and A via matrix/vector multiplication

T = BC*A1

4-element Array{Float64,1}:
 0.47854163222379653
 0.5100446614659566 
 0.564348108912592  
 0.8167190296252382 

In [16]:
# Reshape the result into a matrix (rank-2 tensor)
# T(α*β) --> T(α,β)

T = reshape(ABC1,(Da,Db))

UndefVarError: UndefVarError: ABC1 not defined

### Order of contraction and computational time

The result doesn't change if we change the order with which we contract tensors. However, there might be solutions for which certain contraction scheme speed-up the computation. We will now compare two different possibilities with the precedent tensor network.

In [19]:
# Solution 1 (the one we have already discussed)

# Define the dimensions
Da = 10; # α dimension
Db = 12; # β dimension
Dg = 14; # γ dimension
Dd = 17; # δ dimension
Dm = 20; # μ dimension

# Initialize random matrices
A = rand(Dg,Dd); # Tensor A(γ,δ)
B = rand(Da,Dm,Dg); # Tensor B(α,μ,γ)
C = rand(Db,Dm,Dd); # Tensor C(β,μ,δ)

# Contract
@time begin

    # CONTRACT B and C
    # B(α,μ,γ) -> B(α,γ,μ)
    B1 = permutedims(B,[1,3,2]);
    # B(α,γ,μ) --> B(α*γ,μ)
    B1 = reshape(B1,(Da*Dg,Dm));
    # C(β,μ,δ) -> C(μ,β,δ)
    C1 = permutedims(C,[2,1,3]);
    # C(μ,β,δ) --> C(μ,β*δ)
    C1 = reshape(C1,(Dm,Db*Dd));
    # Contract
    BC = B1*C1;
    # BC(α*γ,β*δ) --> BC(α,γ,β,δ)
    BC = reshape(BC,(Da,Dg,Db,Dd));
    
    # CONTRACT BC and A
    # BC(α,γ,β,δ) --> BC(α,β,γ,δ)
    BC = permutedims(BC,[1,3,2,4]);
    # BC(α,β,γ,δ) --> BC(α*β,γ*δ)
    BC = reshape(BC,(Da*Db,Dg*Dd));
    # Reshape A into a vector
    A1 = A[:];
    # Contract
    T = BC*A1;
    # T(α*β) --> T(α,β)
    T = reshape(T,(Da,Db))
    
end

  0.000315 seconds (45 allocations: 505.313 KiB)


10×12 Array{Float64,2}:
 557.62   562.384  552.59   576.619  …  540.375  593.337  597.322  596.675
 553.6    543.857  542.306  568.663     530.995  592.97   590.73   584.98 
 542.751  539.837  527.296  551.763     520.361  578.663  565.606  570.479
 568.442  576.59   556.845  586.91      543.296  600.714  608.069  602.737
 579.932  575.129  565.721  594.102     553.978  619.48   616.78   605.504
 526.79   536.037  516.813  545.302  …  510.747  564.558  570.777  564.47 
 572.117  563.396  552.294  582.394     547.217  603.363  610.768  602.605
 555.157  545.932  539.397  562.685     525.443  589.336  591.213  583.974
 512.638  514.219  505.043  529.99      495.154  547.748  547.123  546.594
 557.867  555.03   545.682  572.293     531.388  593.517  592.153  588.874

In [20]:
# Solution 2: first contract A and C, then AC and B

# Define the dimensions
Da = 10; # α dimension
Db = 12; # β dimension
Dg = 14; # γ dimension
Dd = 17; # δ dimension
Dm = 20; # μ dimension

# Initialize random matrices
A = rand(Dg,Dd); # Tensor A(γ,δ)
B = rand(Da,Dm,Dg); # Tensor B(α,μ,γ)
C = rand(Db,Dm,Dd); # Tensor C(β,μ,δ)

# Contract
@time begin

    # CONTRACT A and C
    # C(β,μ,δ) -> C(δ,μ,β)
    C1 = permutedims(C,[3,2,1]);
    # C(δ,μ,β) --> C(δ,μ*β)
    C1 = reshape(C1,(Dd,Dm*Db));
    # Contract
    AC = A*C1;
    # AC(γ,μ*β) --> AC(γ,μ,β)
    AC = reshape(AC,(Dg,Dm,Db));
    
    # CONTRACT AC and A
    # AC(γ,μ,β) --> AC(γ*μ,β)
    AC = reshape(AC,(Dg*Dm,Db));
    # B(α,μ,γ) --> B(α,γ,μ)
    B1 = permutedims(B,[1,3,2])
    # B(α,γ,μ) --> B(α,γ*μ)
    B1 = reshape(B1,(Da,Dm*Dg));
    # Contract
    T = B1*AC;
    # T(α*β) --> T(α,β)
    T = reshape(T,(Da,Db));
    
end

  0.000146 seconds (34 allocations: 82.719 KiB)


10×12 Array{Float64,2}:
 606.233  582.672  644.923  583.293  …  568.783  584.15   607.976  592.566
 653.463  631.278  690.173  626.105     605.109  625.907  649.883  648.001
 611.825  588.412  645.721  587.409     566.52   583.359  607.707  600.158
 613.098  581.764  645.146  581.857     567.241  578.24   605.306  596.54 
 681.56   654.732  718.345  652.464     629.726  649.488  675.983  666.88 
 626.856  598.391  659.692  591.918  …  575.3    592.633  624.273  610.409
 611.253  585.024  648.307  587.941     567.039  581.443  606.749  598.425
 614.047  593.068  651.598  595.44      575.856  589.171  615.007  613.47 
 610.698  578.216  630.339  580.989     561.994  569.982  597.471  588.401
 585.459  559.386  617.024  565.864     540.364  547.059  580.233  571.7  

In [21]:
# Compare results

# Define the dimensions
Da = 10; # α dimension
Db = 12; # β dimension
Dg = 14; # γ dimension
Dd = 17; # δ dimension
Dm = 20; # μ dimension

# Initialize random matrices
A  = rand(Dg,Dd); # Tensor A(γ,δ)
B  = rand(Da,Dm,Dg); # Tensor B(α,μ,γ)
C  = rand(Db,Dm,Dd); # Tensor C(β,μ,δ)

println("Solution 1: Contract B and C, then contract BC and A")
@time begin
    # CONTRACT B and C
    # B(α,μ,γ) -> B(α,γ,μ)
    B1 = permutedims(B,[1,3,2]);
    # B(α,γ,μ) --> B(α*γ,μ)
    B1 = reshape(B1,(Da*Dg,Dm));
    # C(β,μ,δ) -> C(μ,β,δ)
    C1 = permutedims(C,[2,1,3]);
    # C(μ,β,δ) --> C(μ,β*δ)
    C1 = reshape(C1,(Dm,Db*Dd));
    # Contract
    BC = B1*C1;
    # BC(α*γ,β*δ) --> BC(α,γ,β,δ)
    BC = reshape(BC,(Da,Dg,Db,Dd));
    
    # CONTRACT BC and A
    # BC(α,γ,β,δ) --> BC(α,β,γ,δ)
    BC = permutedims(BC,[1,3,2,4]);
    # BC(α,β,γ,δ) --> BC(α*β,γ*δ)
    BC = reshape(BC,(Da*Db,Dg*Dd));
    # Reshape A into a vector
    A1 = A[:];
    # Contract
    T1 = BC*A1;
    # T(α*β) --> T(α,β)
    T1 = reshape(T1,(Da,Db));
end

println("Solution 2: first contract A and C, then AC and B")
@time begin
    # CONTRACT A and C
    # C(β,μ,δ) -> C(δ,μ,β)
    C1 = permutedims(C,[3,2,1]);
    # C(δ,μ,β) --> C(δ,μ*β)
    C1 = reshape(C1,(Dd,Dm*Db));
    # Contract
    AC = A*C1;
    # AC(γ,μ*β) --> AC(γ,μ,β)
    AC = reshape(AC,(Dg,Dm,Db));
    
    # CONTRACT AC and A
    # AC(γ,μ,β) --> AC(γ*μ,β)
    AC = reshape(AC,(Dg*Dm,Db));
    # B(α,μ,γ) --> B(α,γ,μ)
    B1 = permutedims(B,[1,3,2])
    # B(α,γ,μ) --> B(α,γ*μ)
    B1 = reshape(B1,(Da,Dm*Dg));
    # Contract
    T2 = B1*AC;
    # T(α*β) --> T(α,β)
    T2 = reshape(T2,(Da,Db));
end

# The absolute error is given by:
absolute_error = sum(broadcast(abs,T1[:]-T2[:]));
println("The absolute error is: $absolute_error")

relative_error =  sum(broadcast(abs, T1[:]-T2[:])./T1[:])/length(T1);
println("The absolute error is: $relative_error")

Solution 1: Contract B and C, then contract BC and A
  0.000341 seconds (46 allocations: 505.359 KiB)
Solution 2: first contract A and C, then AC and B
  0.000077 seconds (35 allocations: 82.766 KiB)
The absolute error is: 2.114575181622058e-11
The absolute error is: 2.916809284298453e-16


As we can see, the second solution is faster than the first. This is due to computational complexity.
For mor information, refer to the lectures given in the reference. Moreover, they provide the same solution up to the machine precision (~10^16).